# AUDIO WRANGLING 🔊

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from pydub.utils import make_chunks
from pydub import AudioSegment
from scipy.fftpack import fft
from pydub.utils import make_chunks
import numpy as np
import pandas as pd
import re
import pickle

----

In [2]:
path = "../input"
for familia in os.listdir(path):
    familia = os.path.join(path, familia)
    print(familia)

../input/.DS_Store
../input/Bufonidae
../input/Arthroleptidae
../input/Hyperollidae
../input/Leptodactylidae
../input/Dendrobatidae


In [3]:
def audioSegmentData(direc, chunk_length_ms):
    fam = []
    path = f"../input/{direc}"
    for audio in os.listdir(path):
        if audio.endswith('.mp3'):
            croak = AudioSegment.from_mp3(path+"/"+audio)
            croak = croak.set_frame_rate(16000)
            
            croak_chunks = make_chunks(croak, chunk_length_ms)
            fam.append(croak_chunks)
    return fam

---

## Segmenting Recordings

In [4]:
buf = audioSegmentData("Bufonidae", 5000)
buf = [x for e in buf for x in e]
len(buf)

444

In [5]:
den = audioSegmentData("Dendrobatidae", 5000)
den = [x for e in den for x in e]

In [6]:
hyp = audioSegmentData("Hyperollidae", 5000)
hyp = [x for e in hyp for x in e]

In [7]:
lep = audioSegmentData("Leptodactylidae", 5000)
lep = [x for e in lep for x in e]

-------

------

## Extracting Fourier Features

In [8]:
# au.audioFeaturesFourie(buf)

In [8]:
def audioFeaturesFourier(aud):
    arr = aud.get_array_of_samples()[:5120]
    abs_four = np.abs(fft(arr))
    return abs_four

In [9]:
bufFF = [audioFeaturesFourier(chunk) for chunk in buf]
len(bufFF)

444

In [10]:
denFF = [audioFeaturesFourier(chunk) for chunk in den]
len(denFF)

349

In [11]:
hypFF = [audioFeaturesFourier(chunk) for chunk in hyp]
len(hypFF)

275

In [12]:
lepFF = [audioFeaturesFourier(chunk) for chunk in lep]
len(lepFF)

380

### All vectors must have the same length, the shortest of them all is 5120.

In [13]:
x = [len(e) for e in denFF]
print(min(x))
print(max(x))

5120
5120


-----

----

MFCCs se calculan comúnmente de la siguiente forma:1
1. Separar la señal en pequeños tramos.
2. A cada tramo aplicarle la Transformada de Fourier discreta y obtener la potencia espectral de la señal.
3. Aplicar el banco de filtros correspondientes a la Escala Mel al espectro obtenido en el paso anterior y sumar las energías en cada uno de ellos.
4. Tomar el logaritmo de todas las energías de cada frecuencia mel
5. Aplicarle la transformada de coseno discreta a estos logaritmos.

## Calculating the Average Value of each MFCC Coefficient

### Mel-Frequency Cepstral Coefficients

In [10]:
from librosa.feature import mfcc

In [20]:
def mfccCoefficients(sample):
    mfccC = np.mean(mfcc(y=np.array([float(str(e)) for e in sample]), sr=len(sample), n_mfcc=128), axis=1)
    return mfccC

In [21]:
bufccC = mfccCoefficients(buf)
bufccC

ValueError: could not convert string to float: '<pydub.audio_segment.AudioSegment object at 0x11f7d5dc0>'

In [ ]:
denccC = mfccCoefficients(den)
denccC

In [ ]:
hypccC = mfccCoefficients(hyp)
hypccC

In [ ]:
lepccC = mfccCoefficients(lep)
lepccC

In [23]:
# Different voices

In [ ]:
def mfccTransform(arr):
    '''mfcc to all the array (column in this case)'''
    mfc=librosa.feature.mfcc(y=np.array(arr).astype('float64'),sr=24000,n_mfcc=512)
    return np.mean(mfc,axis=1)

In [22]:
# Birdperson

In [ ]:
def mfccCoefficients(sample):
    '''
    Determines the average value of each mfcc coefficient for each window.
    '''
    mfc = np.mean(mfcc(y=np.array([float(e) for e in sample]), sr=len(sample), n_mfcc=128).T, axis=0)
    return mfc

---

## Visualizing the MFCC Series

In [ ]:
mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)

In [ ]:
with plt.xkcd():
    f, ax = plt.subplots(1,4, figsize = (15,4))
    librosa.display.specshow(mfccs, x_axis='time')
    plt.colorbar()
    plt.title('MFCC')
    plt.tight_layout()

In [ ]:
with plt.xkcd():
    f, ax = plt.subplots(1,4, figsize = (15,4))
    librosa.display.specshow(mfccsB, ax = ax[0])
    librosa.display.specshow(mfccsD, ax = ax[1])
    librosa.display.specshow(mfccsH, ax = ax[2])
    librosa.display.specshow(mfccsL, ax = ax[3])
    
    sns.countplot(train_dum['Tag_2'], ax = ax[1])

---

---

## Creating *Familia* Dataframes

In [14]:
buf_or = {"F.Transform": [a for a in bufFF], "Tag": 1}
den_or = {"F.Transform": [a for a in denFF], "Tag": 2}
hyp_or = {"F.Transform": [a for a in hypFF], "Tag": 3}
lep_or = {"F.Transform": [a for a in lepFF], "Tag": 4}

In [16]:
w = pd.DataFrame(buf_or)
x = pd.DataFrame(den_or)
h = pd.DataFrame(hyp_or)
z = pd.DataFrame(lep_or)

totus = pd.concat([w, x, h, z]).reset_index(drop=True)
totus

,F.Transform,Tag
0,"[214.0, 1468.2663139035296, 1354.9901292253478...",1
1,"[2884.0, 28338.15576414532, 62698.33809773314,...",1
2,"[231582.0, 274082.53149146156, 349585.41389145...",1
3,"[79700.0, 102571.46322235715, 153200.915277075...",1
4,"[63870.0, 52903.555252298946, 141644.395098856...",1
...,...,...
1443,"[583.0, 15302.869133939248, 8141.018024000418,...",4
1444,"[7528.0, 36567.48880158042, 32709.202174082795...",4
1445,"[57975.0, 49892.81312863944, 28033.09150727976...",4
1446,"[11773.0, 12848.633686627703, 17174.7713207563...",4


In [22]:
# w.to_pickle('../output/fam_df_buf.csv')

In [23]:
# x.to_pickle('../output/fam_df_den.csv')

In [17]:
# h.to_pickle('../output/fam_df_hyp.csv')

In [25]:
# z.to_pickle('../output/fam_df_lep.csv')

In [ ]:
# totus.to_pickle('../output/fam_df.csv')

## Classifying with get_dummies

In [19]:
dum_totus = pd.get_dummies(totus, prefix=['Tag'], columns=['Tag'])
dum_totus

,F. Transform,Tag_1,Tag_2,Tag_3,Tag_4
0,18001.000000,1,0,0,0
1,2077.438012,1,0,0,0
2,4324.447678,1,0,0,0
3,7462.545756,1,0,0,0
4,8746.229687,1,0,0,0
...,...,...,...,...,...
364827,190248.712618,0,0,0,1
364828,118798.087221,0,0,0,1
364829,129512.736967,0,0,0,1
364830,202574.367594,0,0,0,1


In [20]:
tags = pd.get_dummies(totus["Tags"], prefix=['Tag'])
tags

NameError: name 'totus' is not defined

----

---

## Exporting the df to csv

In [59]:
comolollamo.to_csv('../output/fam_df_real.csv')

In [21]:
# dum_totus.to_csv('../output/fam_dum_df.csv')

---

---

---

# Capeándola

In [17]:
from random import sample 
  
# Prints list of random items of given length 
list1 = [1, 2, 3, 4, 5]  
  
print(sample(list1,3))

[2, 1, 4]


In [21]:
bufFFC = audioFeaturesFourier(buf)
bufFFC = sample(list(bufFFC),30000)
len(bufFFC)

30000

In [22]:
denFFC = audioFeaturesFourier(den)
denFFC = sample(list(denFFC),30000)
len(denFFC)

30000

In [23]:
hypFFC = audioFeaturesFourier(hyp)
hypFFC = sample(list(hypFFC),30000)
len(hypFFC)

30000

In [24]:
lepFFC = audioFeaturesFourier(lep)
lepFFC = sample(list(lepFFC),30000)
len(lepFFC)

30000

In [25]:
buf_orC = {"F. Transform": [a for a in bufFFC], "Tag": 1}
den_orC = {"F. Transform": [a for a in denFFC], "Tag": 2}
hyp_orC = {"F. Transform": [a for a in hypFFC], "Tag": 3}
lep_orC = {"F. Transform": [a for a in lepFFC], "Tag": 4}

In [26]:
wC = pd.DataFrame(buf_orC)
xC = pd.DataFrame(den_orC)
yC = pd.DataFrame(hyp_orC)
zC = pd.DataFrame(lep_orC)

totusC = pd.concat([wC, xC, yC, zC]).reset_index(drop=True)
totusC

,F. Transform,Tag
0,5320.613554,1
1,16340.033255,1
2,5792.903736,1
3,4821.384618,1
4,7837.913872,1
...,...,...
119995,203476.417391,4
119996,191580.512539,4
119997,570034.928791,4
119998,121614.880123,4


In [27]:
#totusC.to_csv('../output/fam_cap_df.csv')